In [1]:
#numeric: pandas and numpy
import numpy as np
import pandas as pd
# graphics
%matplotlib inline 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 

from sklearn import linear_model, svm
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_validate, GridSearchCV

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score,zero_one_loss

from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

## Read Data

In [2]:
name_features = []
with open('sensors-19-05524-s001/sensors-19-05524-s001_new/labels.txt', 'r') as file:
    content = file.read()
    cols = content.split('\n')
    name_features = cols[1:-2]   # remove first column and the last two
    
Y = []
X = []

with open('sensors-19-05524-s001/sensors-19-05524-s001_new/data.txt', 'r') as file:
    content = file.read()
    cols = content.split('\n')
    for c in cols[:-1]:
        data = c.split(',')
        X.append(data[1:-1])
        Y.append(data[-1])
        
# Convert String to Float
X = np.array(X).astype(float)
print(type(X[0][0]))

#Convert String to Int
Y = np.array(Y).astype(int)
print(type(Y[0]))
    
print(f"Features: {len(X[0])}")
print(f"Records in DB: {len(X)}")
print(f"Size Y: {len(Y)}")

<class 'numpy.float64'>
<class 'numpy.int64'>
Features: 533
Records in DB: 4480
Size Y: 4480


## Selection of Features (PCA)

## Training Data

In [3]:
k_fold = StratifiedKFold(n_splits=4)

In [4]:
def processKFold(model, X, Y):
    
    scores = []
    loss = []
    
    for train_indexes, test_indexes in k_fold.split(X,Y):

        # maybe can be need normalize the data
        #scaler = StandardScaler()

        X_train, X_test = X[train_indexes], X[test_indexes]
        Y_train, Y_test = Y[train_indexes], Y[test_indexes]

        #X_train = scaler.fit_transform(X_train)
        #X_test = scaler.transform(X_test)

        model.fit(X_train, Y_train)

        Y_pred = model.predict(X_test)
        
        scores.append(accuracy_score(Y_test, Y_pred))
        loss.append(zero_one_loss(Y_test, Y_pred))
        
    return np.mean(scores), np.mean(loss)

In [5]:
# Multilayer feedforward neural network

# change number after
def mlp30(X, Y):
    
    ppnML = MLPClassifier(hidden_layer_sizes=(30),activation="relu",random_state=1)

    return processKFold(ppnML, X, Y)

In [6]:
# svm rbf and svm linear 

def svmRBF(X, Y):

    svmRBF = svm.SVC(kernel='rbf')
    
    return processKFold(svmRBF, X, Y)

In [7]:
def svmLinear(X, Y):

    svmLinear = svm.SVC(kernel='linear')
    
    return processKFold(svmLinear, X, Y)

In [8]:
acc_mpl, loss_mpl = mlp30(X, Y)

print(f"MPL:\nAcc: {acc_mpl}\nLoss: {loss_mpl}")

Mean acc mpl: (0.3236607142857143, 0.6763392857142857)


In [9]:
acc_svmRBF, loss_svmRBF = svmRBF(X, Y)

print(f"svmRBF:\nAcc: {acc_svmRBF}\nLoss: {loss_svmRBF}")

Mean acc svmRBF: (0.33816964285714285, 0.6618303571428571)


In [ ]:
acc_svmLinear, loss_svmLinear = svmLinear(X, Y)

print(f"svmLinear:\nAcc: {acc_svmLinear}\nLoss: {loss_svmLinear}")